Before we do any work, we need to import several functions from cdapython:
- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents

We're also asking cdapython to report it's version so we can be sure we're using the one we mean to.

In [1]:
from cdapython import Q, columns, unique_terms, query
import cdapython
print(cdapython.__version__)
Q.set_host_url("http://35.192.60.10:8080/")

2022.4.13




CDA data comes from three sources:
- The [Proteomic Data Commons](https://proteomic.datacommons.cancer.gov/pdc/) (PDC)
- The [Genomic Data Commons](https://gdc.cancer.gov/) (GDC)
- The [Imaging Data Commons](https://datacommons.cancer.gov/repository/imaging-data-commons) (IDC)

The CDA makes this data searchable in two ways, by a "Subject" table and by "File" table. 

`Subject` data is information that is intrisic to the individual under study, e.g. `sex`, `race`, `ethnicity`. However any given subject might be part of multiple studies. To make search across datasets easier, the CDA model aggregates this data as `ResearchSubject` information. Subjects that participate in multiple projects (are part of multiple nodes), will have multiple `ResearchSubject` entries.

`Subject` and `ResearchSubject` fields are available for both the "Subject" and "File" tables, however terms specific to files e.g. 'data_type' or 'file_format' are only available in "File" table.

To see what fields are available, we use the command `columns`. Here we're looking at the first ten metadata fields available for files:

In [2]:
columns(files=True, limit=10)

['id',
 'identifier',
 'identifier.system',
 'identifier.value',
 'label',
 'data_category',
 'data_type',
 'file_format',
 'associated_project',
 'drs_uri']

By default, `columns()` returns `Subject` table fields. The first several fields (those without a `.` in them) are `Subject` demographic information, which is intrinsically attached to a given subject. Subsequent entities (i.e. `Research.Subject.xxx`) contain details about specific experiments the subject was part of. They are equivilent to the nodes' `Case` record in the GDC and PDC. 

While available search fields may look like ones you've seen in PDC, GDC or IDC, that does not mean they will contain exactly the same information; several are renamed or restructured in the CDA model. The field name mappings are described in [CDA Schema Field Mapping](../Documentation/Schema.md), but we can also directly get information about what data populates any of these fields using the `unique_terms()` function:

In [ ]:
unique_terms("ResearchSubject.primary_diagnosis_site", files=True, limit=10)

The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---

- **[Q.run()](../../../Documentation/usage/#qrun)** returns **demographic** data for the specified search 
- **[Q.counts()](../../../Documentation/usage/#qcounts)** returns summary information (counts) for the **files** that fit the specified search
- **[Q.files()](../../../Documentation/usage/#qfiles)** returns data for the **files** that fit the specified search
- **[Q.sql()](../../../Documentation/usage/#qsql)** allows you to use SQL syntax instead of Q syntax 
- **[query()](../../../Documentation/usage/#query)** allows you to use a more natural language syntax of Q

---


For todays demo, I'm going to show you how you can use `Q.counts()`, `Q.files()`, and `Q.run()` to build a cohort

## Retrieving summary information

### Demographic summary
Let's run a search with a relatively simple question: We're interested in finding data about Kidney cancer. To run this simple search, we would first construct a query in `Q` and save it to a variable `myquery`:

In [ ]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "Kidney"')

Since we are looking for demographic summary information, we want to use this query in Q.run. We do this by running .run() on the query we just saved, and saving the result to a new variable mydemographic:

In [ ]:
mydemographic = myquery.run()

We're saving information in variables, so we don't get any visible output. To see what our results are, we need to look into the variable. The simplest way is to call `mydemographic` directly:

In [ ]:
mydemographic

This output tells us our QueryID, which we don't really need, but the computer does to track our questions. 
Then it tells us five parameters that describe our results:

---

- **Query:** This is the actual SQL query that was run on our database to retreive your results
- **Offset:** This is how many rows of information we've told the query to skip in the data, here we didn't tell it to skip anything, so the offset is zero
- **Count:** This is how many rows (Subjects) the current page of our results table has. To keep searches fast, we default to pages with 100 rows.
- **Total Row Count:** This is how many rows (Subjects) are in the full results table
- **More pages:** This is alwasys a True or False. False means that our current page has all the availble results. True means that we will see only the first 100 results in this table, and will need to page through for more.

---
    
Now that we've seen the metadata about our results, let's look at the actual table. The easiest way to do this is by using the python function `.to_dataframe()` on our `mydemographic` variable:

In [ ]:
mydemographic.to_dataframe()

#### We can further subset by chaining queries

There are lots of Subjects in our general search for Kidney, so lets try to filter it to just cancer, we're most interested in early stage cancer, so we'll filter by Diagnosis.stage:


In [ ]:
cancerquery = Q('ResearchSubject.Diagnosis.stage = "Stage I"').Or(Q('ResearchSubject.Diagnosis.stage = "Stage II"'))
kidneycancerquery = cancerquery.And(myquery)
kidneycancerdemographic = kidneycancerquery.run()
kidneycancerdemographic

This is a much more managable number of subjects, and now they're targeted by our actual question. Let's peek at the data:

In [ ]:
kidneycancerdemographic.to_dataframe().head()

### File summary
Now that we know there are research subjects that meet our criteria, lets see what data exists about them. We can get a summary of the data for these subjects using the `counts()` feature:

In [ ]:
kidneycancerquery.counts()

### What do these numbers mean?

---
    
- **system:** Which data source contributed this data? The CDA currently has data from IDC, PDC and GDC
- **subject_count:** How many unique individuals meet our query. Note that *within* a data source the number is of *unique* individuals, but the same individuals can have data at multiple centers. Here, there are 371 unique people in the IDC data, however up to 57 of those may be exactly the same people as are in the PDC data.
- **subject_files_count:** This tells you roughly how much data is available. It is the total count of files for all the subjects in `subject_count`, which is also the total number of files that match your search.
- **researchsubject_count:** Some data sources have individual subjects that are in multiple studies, when this happens the individual will have both a "subject" identifier and a "researchsubject" identifier. This column counts the latter. Zero in this column can mean either "there are no research_subjects that meet your search criteria" or "the data source for this row does not create special identifiers for subjects in multiple studies"
- **researchsubject_files_count:** This is the total count of files specific to researchsubjects in `researchsubject_count`. It is a subset of `subject_count`
- **specimen_count:** Some data sources track whether files come from specific specimens from a given individual. This column counts the number of specimens that meet your search criteria. Zero in this column can mean either "there are no specimens that meet your search criteria" or "the data source for this row does not track specimens seperately from subjects"
- **specimen_files_count:** This is the total count of files specific to specimens in `specimen_count`. It is a subset of both `subject_count` and `researchsubject_files_count`

---

## Retrieving data


Let's run the same query, but instead of asking for summary information, lets get the data about each file. We start this process the same way, by making a `Q` statement. We can reuse `cancerquery` here as well, and just run the `.files()` function on it: 

In [ ]:
kidneycancerfiles = kidneycancerquery.files()
kidneycancerfiles

In [ ]:
kidneycancerfiles.to_dataframe()

What are all these fields?

---

- **id:** The unique identifier for this file
- **identifier:** An embedded array of information that includes the originating data center and the ID the file had there
- **label:** The full name of the file
- **data_catagory:** A desecription of the kind of general kind data the file holds
- **data_type:** A more specific descripton of the data type
- **file_format:** The extension of the file
- **associated_project:** The name the data center uses for the study this file was generated for
- **drs_uri:** A unique identifier that can be used to retreive this specific file from a server
- **byte_size:** Size of the file in bytes
- **checksum:** The md5 value for the file
- **data_modality:** A high level descriptor of file data, always one of "Genomic", "Proteomic", or "Imaging"
- **imaging_modality** For files with the `data_modality` of "Imaging", a descriptor for the image type
- **dbgap_accession_number:** An identifier for the dbGaP project this file belongs to
- **Subject:** An embedded array of information that includes the originating data center and the Subject ID the file had there
- **ResearchSubject:** An embedded array of information that includes the originating data center and the ResearchSubject ID the file had there
- **Specimen:** An embedded array of information that includes the originating data center and the Specimen ID the file had there

---
